In [5]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pdb
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Applications/anaconda/envs/insight/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [6]:
import nltk;
from nltk.corpus import stopwords;

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/avichanales/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/avichanales/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/avichanales/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.matutils import sparse2full
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

import spacy
nlp  = spacy.load('en_core_web_md')

## 0.0 Load Data

In [8]:
file_name = '/Users/avichanales/Dropbox/Insight/Project/insight_project/data/interim/charity_data_cleaned_temp.csv'
all_charity = pd.read_csv(file_name)

## 1.0 Pre-process raw text (lamentize and remove stopwords) 

In [9]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            #result.append(lemmatize_stemming(token))
            result.append(token)
            
    return result

def preprocess_docs(docs):
    result = []
    
    for doc in docs:
        result.append(preprocess(doc))
    
    return result

In [10]:
all_charity['description_noname'] = all_charity.apply(lambda x: x['description'].replace(x['name'],""),axis=1)

In [11]:
mission_text = all_charity['description_noname'].astype('str')

#Preprocess mission descriptions
mission_text_pre = preprocess_docs(mission_text)

## 2.0 Convert pre-processed text to vectors and embed

In [13]:
def word_embed_charity_tfidf(processed_docs,word_min=5, word_max_perc=.2):
    
    'Assumes docs have already been pre-processed'
    
    #Create dictionary from corpus
    docs_dict = Dictionary(processed_docs)
    docs_dict.filter_extremes(no_below=word_min, no_above=word_max_perc)
    docs_dict.compactify()
    
    #Convert docs into tf-idf vectors
    docs_corpus = [docs_dict.doc2bow(doc) for doc in processed_docs]
    model_tfidf = TfidfModel(docs_corpus, id2word=docs_dict)
    docs_tfidf  = model_tfidf[docs_corpus]
    docs_vecs   = np.vstack([sparse2full(c, len(docs_dict)) for c in docs_tfidf])
    
    num_docs= np.shape(docs_vecs)[0]
    num_words = np.shape(docs_vecs)[1]

    print("Total # of docs: {}".format(num_docs))
    print("Total # of words in dict: {}".format(num_words))
    
    #For each word in dict obtain embedding vector (Glove vectors)
    tfidf_emb_vecs = np.vstack([nlp(docs_dict[i]).vector for i in range(len(docs_dict))])
    
    # Weight glove vectors by tf-idf values
    docs_emb = np.dot(docs_vecs, tfidf_emb_vecs) 
        
    return docs_emb, docs_dict, model_tfidf, tfidf_emb_vecs

In [14]:
charity_docs_emb, charity_docs_dict, charity_model_tfidf, charity_tfidf_emb_vecs = word_embed_charity_tfidf(mission_text_pre,
                                                                                                            word_min=3, 
                                                                                                            word_max_perc=1)




Total # of docs: 8370
Total # of words in dict: 7995


In [415]:
charity_docs_dict.save_as_text('dict_test')

In [417]:
len(charity_docs_dict)

5080

In [15]:
np.shape(charity_docs_emb)

(8370, 300)

## 4.0 Preprocess Article

In [18]:
file_name = '/Users/avichanales/Dropbox/Insight/Project/raw_data/news_data_all.csv'
all_news = pd.read_csv(file_name)

In [19]:
all_news.head()

,index,id,title,author,date,content,year,month,publication,category,digital,section,url
0,0,1,Agent Cooper in Twin Peaks is the audience: on...,\nTasha Robinson\n,2017-05-31,And never more so than in Showtime’s new...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
1,1,2,"AI, the humanity!",\nSam Byford\n,2017-05-30,AlphaGo’s victory isn’t a defeat for hum...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
2,2,3,The Viral Machine,\nKaitlyn Tiffany\n,2017-05-25,Super Deluxe built a weird internet empi...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
3,3,4,How Anker is beating Apple and Samsung at thei...,\nNick Statt\n,2017-05-22,Steven Yang quit his job at Google in th...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
4,4,5,Tour Black Panther’s reimagined homeland with ...,\nKwame Opam\n,2017-05-15,Ahead of Black Panther’s 2018 theatrical...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN


In [15]:
text= 'PITTSBURGH — Armed with an AR-15-style assault rifle and at least three handguns, a man shouting anti-Semitic slurs opened fire inside a Pittsburgh synagogue Saturday morning, killing at least 11 congregants and wounding four police officers and two others, the authorities said. In a rampage described as among the deadliest against the Jewish community in the United States, the assailant stormed into the Tree of Life Congregation, where worshipers had gathered in separate rooms to celebrate their faith, and shot indiscriminately into the crowd, shattering what had otherwise been a peaceful morning. The assailant, identified by law enforcement officials as Robert D. Bowers, fired for several minutes and was leaving the synagogue when officers, dressed in tactical gear and armed with rifles, met him at the door. According to the police, Mr. Bowers exchanged gunfire with officers before retreating back inside and barricading himself inside a third-floor room. He eventually surrendered. Mr. Bowers, 46, was injured by gunfire, although the authorities said it was unclear whether those wounds were self-inflicted or whether the police had shot him. He was taken to Allegheny General Hospital. Federal officials charged Mr. Bowers with 29 criminal counts. They included obstructing the free exercise of religious beliefs — a hate crime — and using a firearm to commit murder. He also faces state charges, including 11 counts of criminal homicide, six counts of aggravated assault and 13 counts of ethnic intimidation. The authorities said that he had no previous criminal history.Though a bris, a ceremony to mark a child’s birth, was among the ceremonies taking place Saturday, no children were among the casualties, law enforcement officials said. The wounded included a 70-year-old man who had been shot in the torso, and a 61-year-old woman with soft tissue wounds, said Dr. Donald Yealy, chairman of emergency medicine at the University of Pittsburgh School of Medicine. The attack Saturday morning struck the heart of the city’s vibrant Jewish community, in the leafy Squirrel Hill neighborhood that is home to several synagogues, kosher restaurants and bakeries. Hours later, hundreds gathered at three separate interfaith vigils on a cold, rainy evening to mourn the dead and pray for the wounded. The assault on the synagogue unfolded on a quiet, drizzly morning, and came amid a bitter, vitriolic midterm election season and against the backdrop of what appears to be a surge in hate-related speech and crimes across America. It also took place in the wake of the arrest Friday morning of a man who the authorities said sent more than a dozen pipe bombs to critics of Mr. Trump, including several high-profile Democrats. Calling it the  most horrific crime scene  he had seen in 22 years with the F.B.I., Robert Jones, special agent in charge in Pittsburgh, said the synagogue was in the midst of a  peaceful service  when congregants were gunned down and  brutally murdered by a gunman targeting them simply because of their faith.   We simply cannot accept this violence as a normal part of American life,  said Gov. Tom Wolf of Pennsylvania, speaking at a news conference Saturday afternoon in Pittsburgh.  These senseless acts of violence are not who we are as Pennsylvanians and are not who we are as Americans.  The anguish of Saturday’s massacre heightened a sense of national unease over increasingly hostile political rhetoric. Critics of President Trump have argued that he is partly to blame for recent acts of violence because he has been stirring the pot of nationalism, on Twitter and at his rallies, charges that Mr. Trump has denied. About Saturday’s attack, Mr. Trump, addressing reporters at Joint Base Andrews, said:  It’s a terrible, terrible thing what’s going on with hate in our country and frankly all over the world, and something has to be done.   The results are very devastating,  he said, adding that if the temple  had some kind of protection  then  it could have been a much different situation.  Later, speaking to reporters as he got off Air Force One in Illinois, Mr. Trump said he planned to visit Pittsburgh but he did not say when. Leaders in the United States and across the world condemned the attack. Prime Minister Benjamin Netanyahu of Israel said he was  heartbroken and appalled  and that the  the entire people of Israel grieve with the families of the dead.  Attorney General Jeff Sessions said that criminal charges by the Justice Department  could lead to the death penalty.   Hatred and violence on the basis of religion can have no place in our society,  Mr. Sessions said.  Every American has the right to attend their house of worship in safety.  The massacre Saturday was at least the third mass shooting in a house of worship in three years. Last November, a gunman killed 26 worshipers at a church in Sutherland Springs, Tex., and in 2015, a white supremacist killed nine congregants in a church in Charleston, S.C. It came amid rising anxiety about illegal immigration and in a decade that has seen an uptick in hate crimes. According to an annual report by the Anti-Defamation League issued earlier this year, the number of reported anti-Semitic incidents in the United States surged 57 percent in 2017, the largest rise in a single year since the A.D.L. began tracking such crimes in 1979. The attack also was a deep and painful blow to the Jewish community in the United States, and came just days after George Soros, the billionaire philanthropist and major donor to Democratic candidates, who is Jewish and who survived Nazi occupation in Hungary, received a pipe bomb in the mail. Also in the past week, a Senate campaign sign for Josh Hawley, attorney general of Missouri, was sprayed with a swastika.'

In [125]:
def df_topic_articles(df, search_term):
    df['topic'] = all_news['content'].str.find(search_term)
    df_sub = df[df['topic']>-1]
    articles = df_sub['content'].astype('str')
    return articles

In [154]:
def df_topic_headlines(df, search_term):
    df['topic'] = all_news['title'].str.find(search_term)
    df_sub = df[df['topic']>-1]
    return df_sub

In [118]:
cc = df_topic_articles(all_news,'climate change')
homeless = df_topic_articles(all_news,'homeless')
jewish = df_topic(all_news,'jew')

NameError: name 'df_topic' is not defined

In [155]:
cc_headline = df_topic_headlines(all_news,'climate change')
homeless_headline = df_topic_headlines(all_news,'homeless')
jewish_headline = df_topic_headlines(all_news,'jew')

In [201]:
headline = jewish_headline.iloc[1,:]
headline['title']

'Andrzejewski: Can Sanctuary Cities Violate Federal Law with No Consequences?'

In [187]:
del all_news

In [20]:
def word_embed_articles(processed_docs,doc_dict,model_tfidf,tfidf_emb_vecs):
    
    #Convert docs into tf-idf vectors
    docs_corpus = [doc_dict.doc2bow(doc) for doc in processed_docs]
    docs_tfidf  = model_tfidf[docs_corpus]
    docs_vecs   = np.vstack([sparse2full(c, len(doc_dict)) for c in docs_tfidf])
    
    
    # sum of glove vectors linearlly weighted by tfidf 
    art_emb = np.dot(docs_vecs, tfidf_emb_vecs)
    
    return art_emb

In [21]:
def process_predict_articles(df, model,column,charity_docs_dict,charity_model_tfidf,charity_tfidf_emb_vecs):
    
    df_content = [df[column]]
    df_pre = preprocess_docs(df_content)
                    
    art_embs = word_embed_articles(df_pre,charity_docs_dict,charity_model_tfidf,charity_tfidf_emb_vecs)
    
    predictions = model.predict(art_embs)

    df['cluster_predictions'] = predictions
    
    return df
    

In [22]:
def plot_cluster_pred_dist(df):
    
    
    df_prediction_dist = df.groupby('cluster_predictions')['index'].count().reset_index()
    
    sns.catplot(y='index',x='cluster_predictions',kind = 'bar', data = df_prediction_dist)
    
    

In [93]:
def process_embed_text(text,charity_docs_dict,charity_model_tfidf,charity_tfidf_emb_vecs):
    
    text_pre = preprocess(text)
     
    #Convert docs into tf-idf vectors
    doc_corpus = charity_docs_dict.doc2bow(text_pre)
    doc_tfidf  = charity_model_tfidf[doc_corpus]
    doc_vec   = np.vstack([sparse2full(doc_tfidf, len(charity_docs_dict))])
    
    # sum of glove vectors linearlly weighted by tfidf 
    art_emb = np.dot(doc_vec, charity_tfidf_emb_vecs)
    
    
    return art_emb

In [1]:
def compute_similarity_output_n(art_emb,charity_docs_emb,topn):
    
    #compute cosine distance from article embedding to all charities
    sim_to_charities = cosine_similarity(art_emb,charity_docs_emb)
    
    #find topN similarity scores
    sim_scores_sorted = -np.sort(-sim_to_charities).flatten()
    topN_scores = sim_scores_sorted[:topn]
    
    #find topN indices
    indices_sorted = (-sim_to_charities).argsort().flatten()
    topN_indices = indices_sorted[:topn].flatten()
    
    return topN_scores, topN_indices
    
    

In [2]:
def topN_ranked_charities(charity_df, topN_scores, topN_indices):
    
    charity_df_slim = charity_df[['name','subcategory','score','description']]
    
    #Extract topN charities and info
    similar_charities = charity_df_slim.iloc[topN_indices].reset_index()
    
    #Add their similarity scores
    similar_charities['sim_score'] = topN_scores
    
    return similar_charities

In [3]:
text= 'Housing and Urban Development Secretary Ben Carson defended his comments about transgender people this week in an agency-wide email late Friday afternoon, calling press reports of the remarks blatant mischaracterizations. The Washington Post reported Thursday that Carson had expressed concern in a meeting in California about big, hairy men trying to use women’s bathrooms, citing three people present who interpreted the remarks as an attack on transgender women. A leading LGBTQ advocacy group denounced Carson for the comments.'



In [4]:
text = 'poverty'

In [439]:
art_emb = process_embed_text(text,charity_docs_dict,charity_model_tfidf,charity_tfidf_emb_vecs)

In [440]:
topN_scores, topN_indices = compute_similarity_output_n(art_emb,charity_docs_emb,10)
topN_charities = topN_ranked_charities(all_charity, topN_scores, topN_indices)

In [441]:
topN_charities

,index,name,subcategory,score,description,sim_score
0,3244,Save the Family,Homeless Services,81.68,Save the Family equips families to address pov...,0.737888
1,5125,buildOn,Development and Relief Services,93.73,"At home or abroad, buildOn's mission is to bre...",0.734194
2,3971,United Way of Charlotte County,United Ways,89.02,We are leading the united effort to eliminate ...,0.732485
3,4234,World Hope International,Development and Relief Services,92.92,World Hope International is a Christian relief...,0.731553
4,5207,BRAC USA,Development and Relief Services,85.85,The mission of BRAC and BRAC USA is to empower...,0.729067
5,179,Inspiration Corporation,Homeless Services,84.09,"In an atmosphere of dignity and respect, Inspi...",0.722309
6,6105,"Christian Community Action, Connecticut",Homeless Services,87.42,Christian Community Action is a faith-based ec...,0.719255
7,4334,Wilkinson Center,Social Services,89.88,We help clients address barriers to self-suffi...,0.718546
8,4755,American Relief Agency for the Horn of Africa ...,Development and Relief Services,92.74,American Relief Agency for the Horn of Africa'...,0.713041
9,8037,Helping Up Mission,Homeless Services,83.22,Helping Up Mission provides hope to people exp...,0.698258


In [404]:
topN_charities['description'][0].replace('Ministry of Caring',"")

'The  is committed to serving the ongoing needs of the poor: food for the hungry, shelter, transitional living, and permanent housing for the homeless, jobs for the unemployed, clothing and furniture for the needy, child care for homeless children, medical and dental services for the poor, and advocacy as well as outreach for the disenfranchised. It is our belief that the poor should never be served poorly, but with love, dignity and respect. All  programs offer hospitality and friendship, while striving to meet basic needs, and help the poor attain self-sufficiency. We minister to the poor by offering them not only the necessities of life- food, shelter, and employment- but also by helping restore their sense of self worth and hope for the future.'

In [351]:
mission_text_pre[7286]

['focus',
 'renew',
 'sto',
 'rox',
 'neighborhood',
 'corpor',
 'short',
 'found',
 'founder',
 'lead',
 'group',
 'mobil',
 'citizen',
 'communiti',
 'activist',
 'corrupt',
 'local',
 'politician',
 'creat',
 'umbrella',
 'organ',
 'need',
 'social',
 'servic',
 'rang',
 'food',
 'secur',
 'nutrit',
 'earli',
 'childhood',
 'educ',
 'support',
 'group',
 'mental',
 'health',
 'art',
 'program',
 'age',
 'small',
 'busi',
 'incub',
 'grow',
 'hous',
 'staff',
 'locat',
 'mckee',
 'rock',
 'stow',
 'township',
 'aim',
 'support',
 'peopl',
 'communiti',
 'program',
 'age',
 'aim',
 'develop',
 'healthi',
 'mind',
 'bodi',
 'spirit',
 'work',
 'wide',
 'rang',
 'partner',
 'ensur',
 'high',
 'qualiti',
 'program',
 'access',
 'resid',
 'sto',
 'rox',
 'communiti']

In [386]:
preprocess('poverty')

['poverti']

In [343]:
def preprocess_pos(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [347]:
preprocess_pos(text)

[('A', 'DT'),
 ('Pastafarian', 'JJ'),
 ('hilariously', 'RB'),
 ('trolled', 'VBD'),
 ('a', 'DT'),
 ('town', 'NN'),
 ('council', 'NN'),
 ('meeting', 'NN'),
 ('.', '.'),
 ('The', 'DT'),
 ('stakes', 'NNS'),
 ('are', 'VBP'),
 ('profoundly', 'RB'),
 ('serious', 'JJ'),
 ('.', '.')]

In [350]:
preprocess_pos(topN_charities['description'][1])

[('The', 'DT'),
 ('Focus', 'NNP'),
 ('On', 'IN'),
 ('Renewal', 'NNP'),
 ('Sto-Rox', 'NNP'),
 ('Neighborhood', 'NNP'),
 ('Corporation', 'NNP'),
 (',', ','),
 ('or', 'CC'),
 ('FOR', 'NNP'),
 ('for', 'IN'),
 ('short', 'JJ'),
 (',', ','),
 ('was', 'VBD'),
 ('founded', 'VBN'),
 ('in', 'IN'),
 ('1969', 'CD'),
 ('.', '.'),
 ('FOR', 'IN'),
 ('founders', 'NNS'),
 ('led', 'VBD'),
 ('a', 'DT'),
 ('group', 'NN'),
 ('of', 'IN'),
 ('mobilized', 'VBN'),
 ('citizens', 'NNS'),
 ('and', 'CC'),
 ('community', 'NN'),
 ('activists', 'NNS'),
 ('against', 'IN'),
 ('corrupt', 'JJ'),
 ('local', 'JJ'),
 ('politicians', 'NNS'),
 ('to', 'TO'),
 ('create', 'VB'),
 ('an', 'DT'),
 ('umbrella', 'JJ'),
 ('organization', 'NN'),
 ('for', 'IN'),
 ('much', 'JJ'),
 ('needed', 'VBN'),
 ('social', 'JJ'),
 ('services', 'NNS'),
 ('now', 'RB'),
 ('ranging', 'VBG'),
 ('from', 'IN'),
 ('food', 'NN'),
 ('security', 'NN'),
 (',', ','),
 ('nutrition', 'NN'),
 (',', ','),
 ('early-childhood', 'JJ'),
 ('education', 'NN'),
 (',', ','),